In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pygrib
from netCDF4 import Dataset

In [ ]:
path_to_model = "/mnt/data-weather/moddata/modgrb/"
path_to_satellite = "/mnt/data-weather/satdata/satnc/band2346/"
path_to_radar = "/mnt/data-weather/raddata/raddat/"

In [ ]:
path_to_model_output = "/mnt/data-weather/moddata/modpng/"
path_to_satellite_output = "/mnt/data-weather/satdata/satpng/band2346/"
path_to_radar_output = "/mnt/data-weather/raddata/radpng/"

In [2]:
path_to_satellite = "/mnt/data-weather/corr/sat/band3/"
path_to_satellite_output = "/mnt/data-weather/corr/A3/"
directory = os.fsencode(path_to_satellite)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    #print(filename)
    nc = Dataset(path_to_satellite+filename, 'r')
    #extracting the data we want from the file
    data = nc.variables['data'][:][0]
    lat = nc.variables['lat'][:]
    lon = nc.variables['lon'][:]
    nc.close()
    #print(data.shape, lat.shape, lon.shape)
    plt.figure(figsize=(10,10))
    m = Basemap(projection='mill',
            resolution=None,
            llcrnrlon=-102,
            urcrnrlon=-82,
            llcrnrlat=31,
            urcrnrlat=41)
    x,y = m(lon,lat)
    cs = m.pcolormesh(x,y,data,shading='flat',cmap="gray")
    plt.savefig(path_to_satellite_output+filename[0:-3]+".png",
                bbox_inches='tight',
                pad_inches=-0.08)
    plt.clf()
    plt.close()

In [ ]:
import os
from shutil import copy2
path_to_satellite = "/mnt/data-weather/satdata/satnc/band2346/"
path_to_other = "/mnt/data-weather/satdata/satnc/other/"
directory = os.fsencode(path_to_satellite)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    #print(filename[18:20])
    if filename[18:20] == "15":
        copy2(path_to_satellite+filename, path_to_other+filename)

In [ ]:
from datetime import timedelta
import datetime
directory = os.fsencode(path_to_other)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print(filename)
    print(filename[12:15])
    date = datetime.datetime(2017, 1, 1) + datetime.timedelta(int(filename[12:15]) - 1)
    if (date.month <= 9 and date.day <= 9):
        #print("sat."+str(date.year)+"0"+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+"0"+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    elif (date.month <= 9 and date.day > 9):
        #print("sat."+str(date.year)+"0"+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+"0"+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    elif (date.month > 9 and date.day <= 9):
        #print("sat."+str(date.year)+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    else:
        #print("sat."+str(date.year)+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    os.rename(path_to_other+filename, path_to_other+name)

In [ ]:
sat = "/mnt/data-weather/satdata/satnc/other/"
rad = "/mnt/data-weather/raddata/raddat/"
sat_out = "/mnt/data-weather/corr/sat/"
rad_out = "/mnt/data-weather/corr/rad/"
sat_dir = os.fsencode(sat)
rad_dir = os.fsencode(rad)

for sat_file in os.listdir(sat_dir):
    sat_filename = os.fsdecode(sat_file)
    for rad_file in os.listdir(rad_dir):
        rad_filename = os.fsdecode(rad_file)
        #print(sat_filename[8:15], rad_filename[10:17])
        if sat_filename[8:15] == rad_filename[10:17]:
            copy2(sat+sat_filename, sat_out+sat_filename)
            copy2(rad+rad_filename, rad_out+rad_filename)

In [ ]:
sat = "/mnt/data-weather/corr/sat/"
sat_out = "/mnt/data-weather/corr/sat/band"
sat_dir = os.fsencode(sat)
for sat_file in os.listdir(sat_dir):
    file = os.fsdecode(sat_file)
    print(file)
    print(file[17:18])
    if len(file) > 5:
        if int(file[17:18]) == 2:
            os.rename(sat+file,sat_out+"2/"+file)
        if int(file[17:18]) == 3:
            os.rename(sat+file,sat_out+"3/"+file)   
        if int(file[17:18]) == 4:
            os.rename(sat+file,sat_out+"4/"+file)
        if int(file[17:18]) == 6:
            os.rename(sat+file,sat_out+"6/"+file)

In [ ]:
rad = "/mnt/data-weather/corr/rad/"
out = "/mnt/data-weather/corr/B2/"
directory = os.fsencode(rad)
dims = (126,201)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    #print(filename)
    data = np.fromfile(rad+filename,dtype='int16',count=-1,sep='') # 2 byte integers
    #print('size of read vector',len(data))
    data = np.reshape(data,dims)
    #print(data)
    fig = plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(data, cmap='gray', origin='lower')
    fig.savefig(out+filename[0:-4]+".png",
                bbox_inches='tight',
                pad_inches=-0.36)
    plt.close(fig)

In [2]:
from shutil import copy2

In [15]:
rad = "/mnt/data-weather/corr/B2/"
rad_out = "/mnt/data-weather/corr/B/"
sat = "/mnt/data-weather/corr/A2/"
sat_out = "/mnt/data-weather/corr/A/"
rad_dir = os.fsencode(rad)
sat_dir = os.fsencode(sat)
#for file in os.listdir(rad_dir):
    #filename = os.fsdecode(file)
    #print(rad_out+filename[10:14]+filename[15:17]+"B2")
    #copy2(rad+filename,rad_out+filename[10:14]+filename[15:17]+"B2.png")
    
#for file in os.listdir(sat_dir):
    #filename = os.fsdecode(file)
    #print(sat_out+filename[8:12]+filename[13:15]+"A2")
    #copy2(sat+filename,sat_out+filename[8:12]+filename[13:15]+"A2.png")

/mnt/data-weather/corr/A/102903A2
/mnt/data-weather/corr/A/102904A2
/mnt/data-weather/corr/A/102905A2
/mnt/data-weather/corr/A/102906A2
/mnt/data-weather/corr/A/102907A2
/mnt/data-weather/corr/A/102908A2
/mnt/data-weather/corr/A/102909A2
/mnt/data-weather/corr/A/110519A2
/mnt/data-weather/corr/A/110520A2
/mnt/data-weather/corr/A/110521A2
/mnt/data-weather/corr/A/110522A2
/mnt/data-weather/corr/A/110523A2
/mnt/data-weather/corr/A/110600A2
/mnt/data-weather/corr/A/110601A2
/mnt/data-weather/corr/A/110602A2
/mnt/data-weather/corr/A/110603A2
/mnt/data-weather/corr/A/110604A2
/mnt/data-weather/corr/A/110605A2
/mnt/data-weather/corr/A/120500A2
/mnt/data-weather/corr/A/120501A2
/mnt/data-weather/corr/A/120502A2
/mnt/data-weather/corr/A/120503A2
/mnt/data-weather/corr/A/120504A2
/mnt/data-weather/corr/A/120505A2
/mnt/data-weather/corr/A/120817A2
/mnt/data-weather/corr/A/120818A2
/mnt/data-weather/corr/A/120819A2
/mnt/data-weather/corr/A/120820A2
/mnt/data-weather/corr/A/120821A2
/mnt/data-weat

/mnt/data-weather/corr/A/081702A2
/mnt/data-weather/corr/A/081703A2
/mnt/data-weather/corr/A/081817A2
/mnt/data-weather/corr/A/081818A2
/mnt/data-weather/corr/A/081819A2
/mnt/data-weather/corr/A/081820A2
/mnt/data-weather/corr/A/081821A2
/mnt/data-weather/corr/A/081822A2
/mnt/data-weather/corr/A/081823A2
/mnt/data-weather/corr/A/081900A2
/mnt/data-weather/corr/A/081901A2
/mnt/data-weather/corr/A/081902A2
/mnt/data-weather/corr/A/081903A2
/mnt/data-weather/corr/A/082216A2
/mnt/data-weather/corr/A/082217A2
/mnt/data-weather/corr/A/082218A2
/mnt/data-weather/corr/A/082219A2
/mnt/data-weather/corr/A/082220A2
/mnt/data-weather/corr/A/082221A2
/mnt/data-weather/corr/A/082222A2
/mnt/data-weather/corr/A/082223A2
/mnt/data-weather/corr/A/082300A2
/mnt/data-weather/corr/A/082301A2
/mnt/data-weather/corr/A/082314A2
/mnt/data-weather/corr/A/082315A2
/mnt/data-weather/corr/A/082316A2
/mnt/data-weather/corr/A/082317A2
/mnt/data-weather/corr/A/082318A2
/mnt/data-weather/corr/A/082319A2
/mnt/data-weat

/mnt/data-weather/corr/A/070706A2
/mnt/data-weather/corr/A/070716A2
/mnt/data-weather/corr/A/070717A2
/mnt/data-weather/corr/A/070718A2
/mnt/data-weather/corr/A/070719A2
/mnt/data-weather/corr/A/070720A2
/mnt/data-weather/corr/A/070721A2
/mnt/data-weather/corr/A/070722A2
/mnt/data-weather/corr/A/070723A2
/mnt/data-weather/corr/A/070800A2
/mnt/data-weather/corr/A/070801A2
/mnt/data-weather/corr/A/070802A2
/mnt/data-weather/corr/A/070803A2
/mnt/data-weather/corr/A/070919A2
/mnt/data-weather/corr/A/070920A2
/mnt/data-weather/corr/A/070921A2
/mnt/data-weather/corr/A/070922A2
/mnt/data-weather/corr/A/070923A2
/mnt/data-weather/corr/A/071000A2
/mnt/data-weather/corr/A/071001A2
/mnt/data-weather/corr/A/071002A2
/mnt/data-weather/corr/A/071003A2
/mnt/data-weather/corr/A/071004A2
/mnt/data-weather/corr/A/071005A2
/mnt/data-weather/corr/A/071006A2
/mnt/data-weather/corr/A/071007A2
/mnt/data-weather/corr/A/071008A2
/mnt/data-weather/corr/A/071009A2
/mnt/data-weather/corr/A/071010A2
/mnt/data-weat

/mnt/data-weather/corr/A/062112A2
/mnt/data-weather/corr/A/062113A2
/mnt/data-weather/corr/A/062114A2
/mnt/data-weather/corr/A/062115A2
/mnt/data-weather/corr/A/062116A2
/mnt/data-weather/corr/A/062117A2
/mnt/data-weather/corr/A/062118A2
/mnt/data-weather/corr/A/062119A2
/mnt/data-weather/corr/A/062120A2
/mnt/data-weather/corr/A/062121A2
/mnt/data-weather/corr/A/062122A2
/mnt/data-weather/corr/A/062123A2
/mnt/data-weather/corr/A/062200A2
/mnt/data-weather/corr/A/062201A2
/mnt/data-weather/corr/A/062202A2
/mnt/data-weather/corr/A/062203A2
/mnt/data-weather/corr/A/062204A2
/mnt/data-weather/corr/A/062205A2
/mnt/data-weather/corr/A/062206A2
/mnt/data-weather/corr/A/062207A2
/mnt/data-weather/corr/A/062208A2
/mnt/data-weather/corr/A/062209A2
/mnt/data-weather/corr/A/052622A2
/mnt/data-weather/corr/A/052623A2
/mnt/data-weather/corr/A/052700A2
/mnt/data-weather/corr/A/052701A2
/mnt/data-weather/corr/A/052702A2
/mnt/data-weather/corr/A/052703A2
/mnt/data-weather/corr/A/052704A2
/mnt/data-weat

/mnt/data-weather/corr/A/050121A2
/mnt/data-weather/corr/A/050122A2
/mnt/data-weather/corr/A/050123A2
/mnt/data-weather/corr/A/050200A2
/mnt/data-weather/corr/A/050201A2
/mnt/data-weather/corr/A/050202A2
/mnt/data-weather/corr/A/050203A2
/mnt/data-weather/corr/A/050204A2
/mnt/data-weather/corr/A/050205A2
/mnt/data-weather/corr/A/050312A2
/mnt/data-weather/corr/A/050313A2
/mnt/data-weather/corr/A/050314A2
/mnt/data-weather/corr/A/050315A2
/mnt/data-weather/corr/A/050316A2
/mnt/data-weather/corr/A/050317A2
/mnt/data-weather/corr/A/050318A2
/mnt/data-weather/corr/A/050319A2
/mnt/data-weather/corr/A/050320A2
/mnt/data-weather/corr/A/050321A2
/mnt/data-weather/corr/A/050322A2
/mnt/data-weather/corr/A/050323A2
/mnt/data-weather/corr/A/050400A2
/mnt/data-weather/corr/A/050401A2
/mnt/data-weather/corr/A/050402A2
/mnt/data-weather/corr/A/050403A2
/mnt/data-weather/corr/A/050404A2
/mnt/data-weather/corr/A/050405A2
/mnt/data-weather/corr/A/050406A2
/mnt/data-weather/corr/A/050407A2
/mnt/data-weat

/mnt/data-weather/corr/A/041711A2
/mnt/data-weather/corr/A/041712A2
/mnt/data-weather/corr/A/041719A2
/mnt/data-weather/corr/A/041720A2
/mnt/data-weather/corr/A/041721A2
/mnt/data-weather/corr/A/041722A2
/mnt/data-weather/corr/A/041723A2
/mnt/data-weather/corr/A/041800A2
/mnt/data-weather/corr/A/041801A2
/mnt/data-weather/corr/A/041802A2
/mnt/data-weather/corr/A/041803A2
/mnt/data-weather/corr/A/041804A2
/mnt/data-weather/corr/A/041805A2
/mnt/data-weather/corr/A/041914A2
/mnt/data-weather/corr/A/041915A2
/mnt/data-weather/corr/A/041916A2
/mnt/data-weather/corr/A/041917A2
/mnt/data-weather/corr/A/041918A2
/mnt/data-weather/corr/A/041919A2
/mnt/data-weather/corr/A/041920A2
/mnt/data-weather/corr/A/041921A2
/mnt/data-weather/corr/A/041922A2
/mnt/data-weather/corr/A/042000A2
/mnt/data-weather/corr/A/042001A2
/mnt/data-weather/corr/A/042002A2
/mnt/data-weather/corr/A/042003A2
/mnt/data-weather/corr/A/042004A2
/mnt/data-weather/corr/A/042005A2
/mnt/data-weather/corr/A/042006A2
/mnt/data-weat

/mnt/data-weather/corr/A/032504A2
/mnt/data-weather/corr/A/032505A2
/mnt/data-weather/corr/A/032506A2
/mnt/data-weather/corr/A/032619A2
/mnt/data-weather/corr/A/032620A2
/mnt/data-weather/corr/A/032621A2
/mnt/data-weather/corr/A/032622A2
/mnt/data-weather/corr/A/032623A2
/mnt/data-weather/corr/A/032700A2
/mnt/data-weather/corr/A/032701A2
/mnt/data-weather/corr/A/032702A2
/mnt/data-weather/corr/A/032703A2
/mnt/data-weather/corr/A/032716A2
/mnt/data-weather/corr/A/032717A2
/mnt/data-weather/corr/A/032718A2
/mnt/data-weather/corr/A/032719A2
/mnt/data-weather/corr/A/032720A2
/mnt/data-weather/corr/A/032721A2
/mnt/data-weather/corr/A/032722A2
/mnt/data-weather/corr/A/032723A2
/mnt/data-weather/corr/A/032800A2
/mnt/data-weather/corr/A/032801A2
/mnt/data-weather/corr/A/032802A2
/mnt/data-weather/corr/A/032818A2
/mnt/data-weather/corr/A/032819A2
/mnt/data-weather/corr/A/032820A2
/mnt/data-weather/corr/A/032821A2
/mnt/data-weather/corr/A/032822A2
/mnt/data-weather/corr/A/032823A2
/mnt/data-weat

/mnt/data-weather/corr/A/021116A2
/mnt/data-weather/corr/A/021120A2
/mnt/data-weather/corr/A/021123A2
/mnt/data-weather/corr/A/021200A2
/mnt/data-weather/corr/A/021201A2
/mnt/data-weather/corr/A/021202A2
/mnt/data-weather/corr/A/021203A2
/mnt/data-weather/corr/A/021204A2
/mnt/data-weather/corr/A/021205A2
/mnt/data-weather/corr/A/021206A2
/mnt/data-weather/corr/A/021412A2
/mnt/data-weather/corr/A/021413A2
/mnt/data-weather/corr/A/021414A2
/mnt/data-weather/corr/A/021415A2
/mnt/data-weather/corr/A/021416A2
/mnt/data-weather/corr/A/021417A2
/mnt/data-weather/corr/A/021418A2
/mnt/data-weather/corr/A/021419A2
/mnt/data-weather/corr/A/021420A2
/mnt/data-weather/corr/A/021421A2
/mnt/data-weather/corr/A/021422A2
/mnt/data-weather/corr/A/021423A2
/mnt/data-weather/corr/A/021500A2
/mnt/data-weather/corr/A/022315A2
